In [ ]:
# import os
# import zipfile

# local_zip = '/content/drive/MyDrive/image/cats_and_dogs_filtered.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/drive/MyDrive/image/cats_dogs')
# zip_ref.close()

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# '/content/drive/MyDrive/image/cats_dogs/cats_dogs_filtered/train/cats'
# '/content/drive/MyDrive/image/cats_dogs/cats_dogs_filtered/train/dogs'
# '/content/drive/MyDrive/image/cats_dogs/cats_dogs_filtered/validation/cats'
# '/content/drive/MyDrive/image/cats_dogs/cats_dogs_filtered/validation/dogs'

In [ ]:
import glob
import numpy as np

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split

In [ ]:
train_photo = list()
train_label = list()
validation_photo = list()
validation_label = list()
for file in glob.glob(f"/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/train/dogs/*"):
    output = 1.0
    photo=load_img(file,grayscale=True,target_size=(150, 150))
    photo = img_to_array(photo)
    
    train_photo.append(photo)
    train_label.append(output)

for file in glob.glob(f"/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/train/cats/*"):
    output = 0.0
    photo=load_img(file,grayscale=True,target_size=(150, 150))
    photo = img_to_array(photo)
    
    train_photo.append(photo)
    train_label.append(output)
train_photo = np.asarray(train_photo)
train_label = np.asarray(train_label)
print(train_photo.shape)
print(train_label.shape)

In [ ]:
for file in glob.glob(f"/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/validation/dogs/*"):
    output = 1.0
    photo=load_img(file,grayscale=True,target_size=(150, 150))
    photo = img_to_array(photo)
    
    validation_photo.append(photo)
    validation_label.append(output)

for file in glob.glob(f"/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/validation/cats/*"):
    output = 0.0
    photo=load_img(file,grayscale=True,target_size=(150, 150))
    photo = img_to_array(photo)
    
    validation_photo.append(photo)
    validation_label.append(output)
validation_photo = np.asarray(validation_photo)
validation_label = np.asarray(validation_label)
print(validation_photo.shape)
print(validation_label.shape)

In [ ]:
num_labels=len(np.unique(train_label))
train_label=to_categorical(train_label)
from keras.layers import normalization
image_size=train_photo.shape[1]
train_photo=np.reshape(train_photo,[-1,image_size,image_size,1])
validation_photo=np.reshape(validation_photo,[-1,image_size,image_size,1])
train_photo=train_photo.astype('float32')/255
validation_photo=validation_photo.astype('float32')/255

X_train, X_test, y_train, y_test = train_test_split(train_photo, train_label, test_size=0.33, random_state=42)

input_shape=(image_size,image_size,1)
batch_size=200
kernel_size=3
pool_size=2
filters=64
dropout=0.5

In [ ]:
model=Sequential()
model.add(Conv2D(filters=filters, 
                  kernel_size=kernel_size, 
                  activation='relu',
                  input_shape=input_shape,
                  padding='same'))
model.add(Conv2D(filters=filters, 
                  kernel_size=kernel_size, 
                  activation='relu',
                  input_shape=input_shape,
                  padding='same'
                  ))
#model.add(normalization.BatchNormalization())
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters*2, 
                  kernel_size=kernel_size, 
                  activation='relu',
                  input_shape=input_shape,
                  padding='same'
                  ))
model.add(Conv2D(filters=filters*2, 
                  kernel_size=kernel_size, 
                  activation='relu',
                  input_shape=input_shape,
                  padding='same'
                  ))
#model.add(normalization.BatchNormalization())
model.add(MaxPooling2D(pool_size))

model.add(Flatten())
model.add(Dropout(dropout))
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(2, activation='sigmoid'))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=50,batch_size=batch_size)
loss,acc=model.evaluate(X_test,y_test,batch_size=batch_size)
print("\naccuracy:%.1f%%\n"%(100.0*acc))

In [ ]:
prediction = np.argmax(model.predict(validation_photo), axis=-1)

import pandas as pd
confussion_matrix=pd.crosstab(validation_label,prediction,rownames=['label'],colnames=['predict'])
print(confussion_matrix)
print("prediction accuracy:",sum(prediction==validation_label)/len(validation_label))

In [ ]:
from matplotlib import pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")